# IGTI - Cientista de Dados - Módulo 2
## Desafio - Parte 3

In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-3.1.2-bin-hadoop2.7'

In [2]:
import pyspark
from pyspark import SparkContext
sc = SparkContext()

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import desc
import pandas as pd

In [4]:
spark = SparkSession.builder.getOrCreate()

In [7]:
# Importar dataset
df = spark.read.csv('stroke_data.csv', inferSchema=True, header=True)
df.show(3)
#df.head()

+---+------+----+------------+-------------+------------+---------+--------------+-----------------+-----+--------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|  bmi|smoking_status|stroke|
+---+------+----+------------+-------------+------------+---------+--------------+-----------------+-----+--------------+------+
|  1|Female|18.0|           0|            0|          No|  Private|         Urban|            94.19|12.12|        smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|  Private|         Rural|           154.24| 33.7|  never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes| Govt_job|         Urban|            72.63| 24.7|        smokes|     0|
+---+------+----+------------+-------------+------------+---------+--------------+-----------------+-----+--------------+------+
only showing top 3 rows



In [8]:
# cria indexers e encoders para variaveis nao numericas

gender_indexer = StringIndexer(inputCol='gender', outputCol='gender_index')
smoking_status_indexer = StringIndexer(inputCol='smoking_status', outputCol='smoking_status_index')

gender_encoder = OneHotEncoder(inputCol='gender_index', outputCol='gender_vector')
smoking_status_encoder = OneHotEncoder(inputCol='smoking_status_index', outputCol='smoking_status_vector')

In [9]:
# transformer para criacao de uma nova coluna 'features' que corresponde a 
# um vetor das variaveis selecionadas (atentar que para as variaveis nao numericas
# sao utilizadas as novas varivaeis criadas com indexers e encoders)
assembler = VectorAssembler(inputCols=['gender_vector', 'smoking_status_vector', 'bmi', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level'], outputCol='features')

In [10]:
# cria um classifier informando qual a variavel alvo e 
# qual a variavel que corresponde ao vetor das outras variaveis selecionadas
classifier = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

In [11]:
# cria o pipeline
pipeline = Pipeline(stages=[gender_indexer, smoking_status_indexer, gender_encoder, smoking_status_encoder, assembler, classifier])

In [12]:
# separa amostras para treinar o modelo e para realizar testes
training_df, test_df = df.randomSplit([0.7, 0.3])

In [13]:
# treina o modelo a partir da amostra separada para treino
model = pipeline.fit(training_df)

In [14]:
# exibe o algoritmo do modelo
# print(model.stages[5].toDebugString)

In [15]:
# realiza predicoes para a amostra separada para testes utilizando o modelo criado
predictions = model.transform(test_df)

In [16]:
# cria um evaluator para avaliar o modelo utilizando a metrica 'accuracy'
eval_acc = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction', metricName='accuracy')

In [17]:
# calcula a acuracia das predicoes realizadas
accuracy = eval_acc.evaluate(predictions)
accuracy

0.835845978943206

In [18]:
# cria um evaluator para avaliar o modelo utilizando a metrica 'weightedPrecision'
eval_prec = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction', metricName='weightedPrecision')

In [19]:
precision = eval_prec.evaluate(predictions)
precision

0.8502742469040172

### Perguntas

**12) Ajuste um modelo de árvore de decisão a partir de pelo menos as seguintes variáveis: gênero, fuma / não fuma, BMI, idade, hipertensão, doença do coração, nível médio de glicose.   
A acurácia deste modelo ficou entre:**

In [16]:
print(f'Acuracia: {round(accuracy * 100)}%')

Acuracia: 83%


**13) Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu na questão 12?**

In [17]:
# Funcao que retorna um dataframe do pandas com os nomes das variaveis ordenados pelo score de importancia
# Fonte: https://www.timlrx.com/blog/feature-selection-using-feature-importance-score-creating-a-pyspark-estimator

def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [18]:
model.stages[-1].featureImportances

SparseVector(9, {2: 0.4975, 3: 0.3406, 4: 0.0011, 5: 0.1604, 8: 0.0004})

In [19]:
ExtractFeatureImp(model.stages[-1].featureImportances, predictions, "features").head(10)

# R: smoking_status

,idx,name,score
7,2,smoking_status_vector_smokes,0.497541
8,3,smoking_status_vector_formerly smoked,0.340617
1,5,age,0.160361
0,4,bmi,0.001078
4,8,avg_glucose_level,0.000403
2,6,hypertension,0.000000
3,7,heart_disease,0.000000
5,0,gender_vector_Female,0.000000
6,1,gender_vector_Male,0.000000


**14) Qual a profundidade da árvore de decisão da questão 12?**

In [20]:
model.stages[-1].depth

5

**15) Quantos atributos foram considerados na árvore?**

In [21]:
model.stages[-1].numFeatures

9